In [2]:
import json
import numpy as np
import torch
import scipy.sparse as sparse
from ogb.graphproppred import GraphPropPredDataset
from torch_geometric.loader import DataLoader

# Download and process data at './dataset/ogbg_code2/'
dataset = GraphPropPredDataset(name = 'ogbg-code2')


split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]


Using backend: pytorch


In [ ]:
print(dataset[0][0].keys())

In [4]:
from tqdm import tqdm
node_list = []
labels = []
edges = []
sizes=[]
node_depth=[]
edge_feats = []
node_feats = []
num_nodes = 0

In [5]:
for g, label in tqdm(dataset):
    node_list.append(np.arange(g["num_nodes"]) + num_nodes)  # All the nodes are considered in a single graph
    edges.append(np.stack(g["edge_index"]).T + num_nodes)
    edge_feats.append(g["edge_feat"])
    node_feats.append(g["node_feat"])
    while len(label) < 19:
        label.append("")
    labels.append(label)
    sizes.append(len(label))
    num_nodes += g["num_nodes"]
print(len(edges))
print(num_nodes)



100%|██████████| 452741/452741 [00:11<00:00, 38735.03it/s]

452741
56688578


In [6]:
print(np.size(edge_feats))
print(edge_feats[333])


452741
None


In [7]:
_edges = np.concatenate(edges, axis=0)
_labels = np.stack(labels).squeeze()
_node_feats = np.concatenate(node_feats)

In [8]:
print(len(sizes))
print(np.max(sizes))
print(np.average(sizes))

452741
19
19.0


In [9]:
_node_list = sparse.lil_matrix((len(dataset), num_nodes))
for i, indices in enumerate(node_list):
    _node_list[i, indices] = 1

In [11]:
_edges = np.concatenate(edges, axis=0)
_labels = np.stack(labels).squeeze()
_node_feats = np.concatenate(node_feats)

In [12]:
data = {
    "node_feats": _node_feats,
    "edge": _edges,
    "graph_class": _labels,
    "node_list": _node_list,
}
np.savez_compressed("ogbg-molfreesolv.npz", **data)

In [1]:
task_data={
    "train" :train_idx,
    "val" :valid_idx,
    "test" :test_idx
}
np.savez_compressed("ogbg-molpcba_task.npz",**task_data)

NameError: name 'train_idx' is not defined